# Convert PicoQuant ns-ALEX files to Photon-HDF5

<p class="lead">This <a href="https://jupyter.org/">Jupyter notebook</a>
will guide you through the conversion of a ns-ALEX data file from PicoQuant formats <b>(PTU, HT3, PT3)</b>
to <a href="http://photon-hdf5.org">Photon-HDF5</a> format. For more info on how to edit
a jupyter notebook refer to <a href="http://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Notebook%20Basics.ipynb#Overview-of-the-Notebook-UI">this example</a>.</p>

*Please send feedback and report any problem to the 
[Photon-HDF5 google group](https://groups.google.com/forum/#!forum/photon-hdf5).*

# 1. How to run it?

The notebook is composed by "text cells", such as this paragraph, and "code cells"
containing the code to be executed (and identified by an `In [ ]` prompt). 
To execute a code cell, select it and press **SHIFT+ENTER**. 
To modify an cell, click on it to enter "edit mode" (indicated by a green frame), 
then type.

You can run this notebook directly online (for demo purposes), or you can 
run it on your on desktop. For a local installation please refer to:

- [Jupyter Notebook Quick-Start Guide](http://jupyter-notebook-beginner-guide.readthedocs.org) 

<br>
<div class="alert alert-info">
Please run each each code cell using <b>SHIFT+ENTER</b>.
</div>

# 2. Prepare the data file

## 2.1 Import modules

First we need to import ``phconvert`` as well as a few other standard python modules into the python environment

In [ ]:
import os
import numpy as np

import phconvert as phc
print('phconvert version: ' + phc.__version__)

## 2.2 Upload the data file

<br>
<div class="alert alert-info">
<b>Note:</b> if you are running the notebook locally skip to section [<b>2.3</b>].
</div>

Before starting, you have to upload a data file to be converted to Photon-HDF5.
You can use one of our example data files available
[on figshare](http://dx.doi.org/10.6084/m9.figshare.1455963). 

To upload a file (up to 35 MB) switch to the "Home" tab in your browser, 
click the **Upload** button and select the data file
and wait until the upload completes.
To upload files larger than 35 MB (like some of our example files) please use the 
[Upload notebook](Upload data files.ipynb) instead.

Once the file is uploaded, come back here and follow the instructions below.

## 2.3 Select the file

Specify the input data file in the following cell:

In [ ]:
filename = 'data/Cy3+Cy5_diff_PIE-FRET.ptu'

The next cell will check if the `filename` location is correct:

In [ ]:
if os.path.isfile(filename):
    print("File found, you can proceed")
else:
    if os.path.exists(filename):
        raise FileNotFoundError(f"'{filename}' is not a file")
    else:
        raise FileNotFoundError(f"'{filename}' does not exist")

If there is an error, adjust the `filename` cell to the correct location

## 3.1 Load the file

Now we can load the data into the notebook

In [ ]:
data, metadata = phc.loader.loadfile_ptu(filename)

There are a number of usual (most of them are **required**) metadata fields in photon-HDF5 that cannot 
be determined from the .ptu file.
Therefore `loader.loadfile_ptu` inserts these fields into the dictionary with `None` values to indicate
to the user that these fields are expected, and must be filled out by the user.

The function `phc.helperfuncs.report_nones` is a convenience function that prints all such `None` fields
in a dictionary.

In [ ]:
phc.helperfuncs.report_nones(data)

#### 3.2.1 Metadata

Now we fill out some metadata so the nature of the experiment can be known.

In [ ]:
# Sample fields
author = 'John Doe'
author_affiliation = 'Research Institution'
description = 'A demonstrative measurement.'
sample_name = 'A demonstrative fluorescently labeled construct'
dye_names = 'dyeA, dyeB'
buffer_name = 'A standard buffer'
measurement_type = 'generic' # can be 'smFRET', 'smFRET-nsALEX', smFRET-usALEX' etc.

sample = dict(
    sample_name=sample_name,
    dye_names=dye_names,
    buffer_name=buffer_name,
    num_dyes = len(dye_names.split(',')))

#### 3.2.2 Emission parameters

As the acquisition software does not know about all the
filters and various distances between detectors etc.,
it is necessary to add these experimental configuration settings
in the jupyter noteboo.

In [ ]:
# Setup parameters
excitation_wavelengths = np.array([488e-9, 632e-9])
excitation_cw = np.array([False, False])
excitation_alternated = np.array([False, False])
detection_wavelengths = np.array([580e-9, 690e-9])

Populate the data dictionary with the various metadata fields in their correct names/positions

In [ ]:
data['description'] = description
data['sample'] = sample
data['identity']['author'] = author
data['identity']['author_affiliation'] = author_affiliation
phc.helperfuncs.fill_measurement_type(data, measurement_type)

The detector indexes used in the file are recorded in the
`/photon_data/measurement_specs/detectors_specs/spectral_polarization_split_chN`
key in the data dictionary.

This key must be removed as it is not a valid field in photon-HDF5.
This field is exported by the `loader.loadfile_ptu` function because
the detectors must be assigned to several field, namely:

- `spectral_ch1`, `spectral_ch2` ... etc
- `polarization_ch1`, `polarization_ch2` ... etc
- `split_ch1`, `split_ch2` ... etc

if for any of these categories has only one channel/photons are not sorted in the
given way by the setup, it can be ommitted.

Since `phconvert` has no way of knowing how to assign detectors to these fields,
it stores all of them in a 'hint' field, so the user can at least know what the
detectors are, and can assign them appropriately.

The cell bellow shows all the detector indexes in a list:

In [ ]:
detectors_specs = data['photon_data']['measurement_specs']['detectors_specs']
setup = data['setup']
detectors_ids = detectors_specs.pop('spectral_polarization_split_chN')
print(detectors_ids)

Now assign the detector ids to the correct channels.

The code below will be modified according to the actual detectors used.

In [ ]:
# Adjust according to how each detector is assigned to spectral/polarization/split channels
detectors_specs['spectral_ch1'] = detectors_ids[:1]
detectors_specs['spectral_ch2'] = detectors_ids[1:]
# detectors_specs['polarization_ch1'] = detectors_ids[::2]
# detectors_specs['polarization_ch2'] = detectors_ids[1::2]
# detectors_specs['split_ch1'] = detectors_ids[::2]
# detectors_specs['split_ch2'] = detectors_ids[1::2]

If your TCSPC device adds sync photons, space-time markers, or any other sort of marker,
these must be assigned to `non_photon_id1` `non_photon_id2` ... etc. (only a single field is needed)

The `loader.loadfile_ptu` loads these into `non_photon_idN` for the same reason it uses
`spectral_polarization_split_chN`. So assign appropriately

If there are no markers, the `non_photon_idN` field will not be present in the output dictionary,
and the `non_photon_id1` field is not needed.

In [ ]:
non_photon = detectors_specs.pop('non_photon_idN', list())
print(non_photon)

The code bellow is a "dummy" code, basically if markers are present, it assigns them all to sequential `non_photon_idN` fields

In [ ]:
if non_photon is not None and len(non_photon):
    for i, nph in enumerate(non_photon):
        detectors_specs[f'non_photon_id{i}'] = np.array([nph,])

### 3.2.3 Emission parameters

When multiple excitations are present, the period for each excitation should be specified.

`phconvert` supplies a useful function for choosing the ideal range(s) for each period,
and for visualizing the appearance of the decays, this is `plotter.alternation_hist`

In [ ]:
phc.plotter.alternation_hist(data, group_dets=True)

From this plot, iterate with the cell below to find the best windows:

In [ ]:
alex_excitation_donor = (50, 850)
alex_excitation_acceptor = (1400, 2250)
phc.helperfuncs.fill_alex_periods(data, alex_excitation_donor, alex_excitation_acceptor)
phc.plotter.alternation_hist(data, group_dets=True)

Once those field look well aligned, there may be an offset between the decays
of different detectors due to speed of light delays.
Therefore, the optional field `/setup/detectors/tcspc_offsets` allows each
detector id to be given its own integer offset, in `tcspc_unit` units.

Again, iterate with the cell below to allign the decays so that they are alligned
within the excitation windows idealy:

In [ ]:
setup['detectors']['tcspc_offsets'] = np.array([0, 75])
phc.plotter.alternation_hist(data, group_dets=True)

Some final fields to fill out:

In [ ]:
phc.helperfuncs.fill_setup(data)

setup['excitation_wavelengths'] = excitation_wavelengths
setup['excitation_cw'] = excitation_cw
setup['excitation_alternated'] = excitation_alternated
setup['detection_wavelengths'] = detection_wavelengths
setup['detectors']['label'] = np.array(['donor','acceptor'])

The `report_nones` function will print out any fields you still need to fill out.
If you have filled everything out, the cell bellow should have no output

In [ ]:
phc.helperfuncs.report_nones(data)

We store the metadata in the `user/picoquant` group so that no data is lost.
This data will not be processed by standard photon-HDF5 readers,
but may be helpful for human readers debugging or othewise investigating the file.

## 4. Save the file

In [ ]:
phc.hdf5.save_photon_hdf5(data, h5_fname='testout.hdf5', overwrite=True)

The conversion is complete, the remainder of the notebook is just for verifying that the file was saved correctly.

## 5. Check file

Now load the file to ch

In [ ]:
h5file = phc.hdf5.load_photon_hdf5('testout.hdf5')
d = phc.hdf5.dict_from_group(h5file.root)
h5file.close()

In [ ]:
d['photon_data']

In [ ]:
d['setup']